In [4]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

In [6]:
def TMensual(Tmort):
   """
   Interpola los valores de l(x) en pasos de 1/12 (mensual) y calcula q(x) y p(x).
   Parameters:
   Tmort (DataFrame): DataFrame que contiene las columnas 'x' y 'l(x)'.
   Returns:
   DataFrame: DataFrame con valores interpolados de x, l(x), q(x) y p(x).
   """
   # Aseguramos que Tmort tiene las columnas adecuadas
   if 'x' not in Tmort.columns or 'l(x)' not in Tmort.columns:
       raise ValueError("El DataFrame debe contener las columnas 'x' y 'l(x)'")
   # Obtener los valores de x y l(x)
   x_values = Tmort['x'].values
   lx_values = Tmort['l(x)'].values
   # Crear una función de interpolación
   interp_function = interp1d(x_values, lx_values, kind='linear', fill_value='extrapolate')
   # Crear los nuevos valores de x con un paso de 1/12, hasta 110 (incluyendo 110)
   new_x = np.arange(x_values[0], 110 + 1/12, 1/12)  # Incrementos mensuales hasta 110
   # Calcular los valores de l(x) interpolados
   new_lx = interp_function(new_x)
   # Inicializar los arrays para q(x) y p(x)
   qx_values = np.zeros(len(new_x) - 1)  # Inicializar el array para q(x)
   px_values = np.zeros(len(new_x) - 1)  # Inicializar el array para p(x)
   # Calcular q(x) y p(x) para cada nuevo valor de x
   for i in range(len(new_x)-1):
       if new_lx[i] > 0:  # Evitar división por cero
           qx_values[i] = 1 - (new_lx[i + 1] / new_lx[i])  # q(x) = 1 - l(x + 1/12) / l(x)
       else:
           qx_values[i] = 0  # Si l(x) es 0, q(x) también debe ser 0
       px_values[i] = 1 - qx_values[i]  # p(x) = 1 - q(x)

   # Crear un nuevo DataFrame con los valores interpolados y calculados
   interpolated_df = pd.DataFrame({
       'x': new_x,  # Incluir hasta 110
       'l(x)': new_lx,  # Excluir el último por la misma razón
       'q(x)': np.append(qx_values, 1),  # Añadir el valor de q(110)
       'p(x)': np.append(px_values, 0)  # Añadir el valor de p(110)
   })
   return interpolated_df
def tmort_filtered(df,edad):
   qx_vec=df[df['x']>=edad]
   return qx_vec